In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize
import requests
import matplotlib.dates as mdates
from datetime import date, timedelta

In [2]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visits = pd.read_sql('SELECT * FROM Visits', con=db_connection)
visits['created'] = pd.to_datetime(visits['created'], unit='s')
visits['lastmodified'] = pd.to_datetime(visits['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['registered'] = visits['registered'].replace(0, np.nan)
visits['registered'] = pd.to_datetime(visits['registered'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['deleted'] = visits['deleted'].replace(0, np.nan)
visits['deleted'] = pd.to_datetime(visits['deleted'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['paired'] = visits['paired'].replace(0, np.nan)
visits['paired'] = pd.to_datetime(visits['paired'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
visits['created_ip'].replace(0, np.nan, inplace=True)
visits['registered_ip'].replace(0, np.nan, inplace=True)
visits['paired_ip'].replace(0, np.nan, inplace=True)
visits['exported'].replace(0, np.nan, inplace=True)
visits['title'].fillna(value=np.nan, inplace=True)
visits['note'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

pensActivity = pd.read_sql('SELECT * FROM PensActivity', con=db_connection)
pensActivity['created'] = pd.to_datetime(pensActivity['created'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensActivity['lastmodified'] = pd.to_datetime(pensActivity['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensActivity['created'].replace(0, np.nan, inplace=True)
pensActivity['lastmodified'].replace(0, np.nan, inplace=True)
pensActivity['notes'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
pensActivity['notes'] = pensActivity['notes'].astype('category')
pensActivity.sort_values(by='created', inplace=True)

In [3]:
visits= visits[visits['created'] > '2014']
visits_opening_hours = visits.set_index('created')
visits_opening_hours = visits_opening_hours.between_time('10:00', '18:00')
visits_opening_hours.reset_index(inplace=True)
pensActivity = pensActivity[pensActivity['created'] > '2014']
activity_openhours = pensActivity.set_index('created')
activity_openhours = activity_openhours.between_time('10:00', '18:00')
activity_openhours.reset_index(inplace=True)

In [8]:
visits_clean = visits_opening_hours[['id', 'shortcode', 'user_id', 'created']]
visits_clean = visits_clean.rename({'id': 'visit_id'},axis=1)
visits_clean

pensActivity_clean = activity_openhours[['visit_id', 'created', 'count_items']]

In [9]:
activity_visits = pensActivity_clean.merge(visits_clean, on='visit_id', how='right', suffixes=('_activity', '_visits'))
activity_visits['count_items'].fillna(value=0, inplace=True)

In [10]:
visitors_collected_something = activity_visits[activity_visits['count_items'] != 0] 
visitors_collected_something['shortcode'].count()

402042

In [11]:
collected_nothing = activity_visits[activity_visits['count_items'] == 0] 
collected_nothing['shortcode'].count()

88047

In [13]:
have_account = activity_visits[activity_visits['user_id'] != 0]
have_account.count()

visit_id            23582
created_activity    23040
count_items         23582
shortcode           23582
user_id             23582
created_visits      23582
dtype: int64

In [17]:
collected_something_and_have_account = visitors_collected_something[visitors_collected_something['user_id'] != 0]
collected_something_and_have_account['shortcode'].count()

23040

In [18]:
have_account['shortcode'].count() / visitors_collected_something['shortcode'].count() * 100

5.865556334910283

In [21]:
did_something_engagment_yearly = collected_something_and_have_account['created_visits'].groupby(collected_something_and_have_account.created_visits.dt.to_period('Y')).agg('count')
did_something_engagment_yearly = did_something_engagment_yearly.to_frame()
did_something_engagment_yearly.rename({'created_visits': 'value'}, axis=1, inplace=True)
did_something_engagment_yearly['total_visits'] = visits['created'].groupby(visits.created.dt.to_period('Y')).agg('count')
did_something_engagment_yearly

In [27]:
did_something_engagment_yearly['percentage'] = collected_something_and_have_account['created_visits'].groupby(collected_something_and_have_account.created_visits.dt.to_period('Y')).agg('count') / visits['created'].groupby(visits.created.dt.to_period('Y')).agg('count') *100
did_something_engagment_yearly= did_something_engagment_yearly[['value', 'total_visits', 'percentage']]
did_something_engagment_yearly.rename({'value': 'total collected AND opened account'}, axis=1, inplace=True)
did_something_engagment_yearly

,total collected AND opened account,total_visits,percentage
created_visits,,,
2015,9408,119850,7.849812
2016,7562,212266,3.562511
2017,3487,140780,2.476914
2018,1393,127028,1.096609
2019,1079,98745,1.092714
2020,111,17426,0.636979
